# Script to merge the prediction of a tabular classifier and a image classifier

In [ ]:
import pandas as pd
import os
import json
import numpy as np
from sklearn import metrics

In [ ]:
tabular_model_name = '{tabular_model}' # INSERT HERE
tabular_exp = 'exp00'
logs_deep = 'logs-{dl_model}' # INSERT HERE
model = logs_deep.split('-')[1]

In [ ]:
tabular_clf_path = os.path.join('..', 'tabular-classifier')
image_clf_path = os.path.join('..', 'image-classifier')

In [ ]:
def get_prediction_image(fold, eco_type, logs_deep):
    path = os.path.join(image_clf_path, logs_deep)
    for exp in os.listdir(path):
        with open(os.path.join(path, exp, 'config.json'), 'r') as file:
            config = json.load(file)
            if config.get('dataset') == f'dataset_images_cv_{fold}/{eco_type}':
                preds = pd.read_csv(os.path.join(path, exp, 'test_prediction.csv'))
                preds['Processo'] = preds.image.str.split('/').apply(lambda x: x[1]).str.split('_').apply(lambda x: x[0]).str.split('PU').apply(lambda x: x[1]).astype(int)
                preds = preds.sort_values(by='Processo')
                preds = preds.set_index('Processo')
                preds = preds.add_prefix(f'{eco_type}_')
                return preds
    return  None

In [ ]:
def get_prediction_tabular(tabular_exp, tabular_model_name, fold):
    tabular_predictions = pd.read_csv(os.path.join(tabular_clf_path, 'logs', tabular_exp, 'predictions', f'{tabular_model_name}_fold{fold}_predictions.csv'))
    tabular_predictions = tabular_predictions.sort_values(by='Processo')
    tabular_predictions = tabular_predictions.set_index('Processo')
    tabular_predictions = tabular_predictions.add_prefix('Tabular_')
    return tabular_predictions

In [ ]:
head_preds = get_prediction_image(eco_type='Head', fold=1, logs_deep=logs_deep)
abd_preds = get_prediction_image(eco_type='Abdomen', fold=1, logs_deep=logs_deep)
femur_preds = get_prediction_image(eco_type='Femur', fold=1, logs_deep=logs_deep)
tabular_preds = get_prediction_tabular(tabular_exp=tabular_exp, tabular_model_name=tabular_model_name, fold=1)
df = pd.concat([tabular_preds, head_preds, abd_preds, femur_preds], axis=1)

In [ ]:
for fold in range(1, 3+1):
    head_preds = get_prediction_image(eco_type='Head', fold=fold, logs_deep=logs_deep)
    abd_preds = get_prediction_image(eco_type='Abdomen', fold=fold, logs_deep=logs_deep)
    femur_preds = get_prediction_image(eco_type='Femur', fold=fold, logs_deep=logs_deep)
    tabular_preds = get_prediction_tabular(tabular_exp=tabular_exp, tabular_model_name=tabular_model_name, fold=fold)
    df = pd.concat([tabular_preds, head_preds, abd_preds, femur_preds], axis=1)
    df.columns = [col.replace('__', '_') for col in df.columns]
    df.to_csv(f'predictions_{tabular_model_name}_{model}_fold_{fold}.csv')